In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# mode
offline_mode = False
anonymous_mode = True
PATH = ''

#imports
from IPython.core.display import HTML, Image, display
from IPython.display import clear_output
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import ipywidgets as widgets
from ipywidgets import Layout, GridBox, Box, VBox, HBox, HTML, Combobox, Dropdown, Select, Button, Label, Textarea, Text, IntProgress, Checkbox
import base64
import json
import os
from urllib.request import urlopen, urlretrieve
from dataclasses import dataclass, field
import codecs
import io
import time
from random import choice
from datetime import datetime
from contextlib import suppress
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
# DB & files
DB_filename = os.path.join(PATH, 'data.json')
cookies_filename = os.path.join(PATH, 'cookies.json')
credentials_filename = os.path.join(PATH, 'credentials.json')
log_filename = os.path.join(PATH, 'journal.log')
export_filename = os.path.join(PATH, 'export.html')
# logging & exceptions
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.remote.remote_connection import LOGGER
import logging
from selenium.webdriver.remote.remote_connection import LOGGER as seleniumLogger
seleniumLogger.setLevel(logging.ERROR)
from urllib3.connectionpool import log as urllibLogger
urllibLogger.setLevel(logging.ERROR)
msg_format = '%(asctime)s - %(levelname)10s: %(message)s'
date_format ='%d.%m.%Y %H:%M:%S'
logging.basicConfig(level=logging.WARNING,
                    format=msg_format,
                    datefmt=date_format,
                    filename=log_filename,
                    filemode='w')
log = logging.getLogger('MambaHunter')
console = logging.StreamHandler()
# log.addHandler(console)
hello_msgs = ['приветствую', 'приветствую Вас', 'привет', 'привет тебе', 'здравствуй', 'здравствуйте', 'добрый день', 'доброго времени суток',
              'салют', 'рад Вас видеть', 'желаю здравствовать', 'моё почтение', 'позвольте Вас приветствовать',
              'познакомимся', 'давай знакомиться', 'давайте знакомиться', 'позвольте с Вами познакомиться']


# main data
data = dict()
# main table categories
categories = ['Входящие', 'Отложенные', 'В работе', 'Корзина']
# driver driver
driver = None
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
authorization_region_lock = True
LANGUAGE = 'ru'
COUNTRY = 'Россия'
CITY = 'Москва, Россия'
# UI
ui = None
outbox = widgets.Output(layout={'border': '1px solid black', 'width':'900px'})
capacity = 2000
progress = IntProgress(min=0, max=capacity, layout=Layout(width='900px'))
current_message_text = ''
profile_cache = []
profile_data_load = True
profile_photos_load = True
profile_history_load = False
guests = dict()

# drive
def drive():
    global driver
    log.debug('starting driver...')
    driver_options = Options()
    driver_options.add_argument(f"--lang={LANGUAGE}")
    os.environ['WDM_LOG_LEVEL'] = '0'
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=driver_options)

    driver.set_window_position(0, 0)
    driver.set_window_size(768, 1024)
    params = dict({
            "latitude": 55.774527,
            "longitude": 37.587093,
            "accuracy": 100
            })
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", params)
    log.debug('driver started...')
#     driver.quit()

# check IP
def check_ip():
    url = 'http://ipinfo.io/json'
    response = urlopen(url)
    data = json.load(response)
    return [data["country"], data["region"], data["city"]]

# authorization
def authorization():
    global driver, cookies_filename, credentials_filename
    log.debug('starting authorization...')
    if authorization_region_lock and check_ip()[0].lower() != LANGUAGE:
        log.warning(f'region is not {LANGUAGE}... authorization canceled')
        return False
    url = f'https://www.mamba.ru/{LANGUAGE}/login'
    login = '';  password = ''
    content = ''
    try:
        if os.path.isfile(credentials_filename):
            with open(credentials_filename) as f:
                try:
                    credentials = json.load(f)
                    login, password = credentials['login'], credentials['password']
                except Exception as e:
                    log.error(f'credentials error: {e}')
                    return False
        if os.path.isfile(cookies_filename):
            log.debug(f'with cookies...')
            driver.get(url)
            with open(cookies_filename, 'r') as f:
                data = json.load(f)
                for item in data:
                    driver.add_cookie(item)
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[@data-name="messenger-action"]')))
        else:
            log.debug(f'without cookies...')
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/input[@type="submit"]')))
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/input[@name="login"]'))).send_keys(login)
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/input[@name="password"]'))).send_keys(password)     
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/input[@type="submit"]'))).click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[@data-name="menu-user_profile-action"]')))
            with open(cookies_filename, 'w') as f:
                json.dump(driver.get_cookies(), f)
        log.debug(f'authorization complete')
        return True
    except Exception as e:
        log.error(f'authorization error: {e}')
        return False
    
# image -> base64
def embeded_image(url):
    try:
        filename = urlretrieve(url)[0]
        data = ''
        with open(filename, 'rb') as f:
            data = base64.b64encode(f.read()).decode('utf-8')
        return f'<img src="data:image/jpeg;base64,{data}" />'
    except:
        return '<img src="data:image/jpeg;base64,iVBORw0KGgoAAAANSUhEUgAAASwAAAGQAQMAAAAuu884AAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAADUExURf///6fEG8gAAAAJcEhZcwAADsMAAA7DAcdvqGQAAAAmSURBVGje7cExAQAAAMKg9U9tDB8gAAAAAAAAAAAAAAAAAACAixo88AABXeKt6QAAAABJRU5ErkJggg==" />'

# move data
def moveto_data(source, destination):
    global data
    log.info(f'data moving from {source} to {destination}...')
    counter = [change_data(k, 'comment', destination) for k, v in data.items() if source in v["comment"]]
    log.info(f'data moved: {source} => {destination}')
    log.debug(f'data moved: {counter} items')
    return counter

# load data
def load_data():
    global data
    log.info(f'data loading from {DB_filename}...')
    if os.path.isfile(DB_filename):
        with open(DB_filename, 'r') as f:
            data = json.load(f)
        log.info(f'data loaded: {DB_filename}')
    else:
        log.warning(f'data not loaded: used blanked data')

#save data
def save_data():
    global data
    log.info(f'data saving to {DB_filename}...')
    with open(DB_filename, 'w') as f:
        json.dump(data, f)
    log.info(f'data saved: {DB_filename}')

# print data
def print_data(category_filter = ''):
    global data, export_filename
    log.info(f'data printing to {export_filename}...')
    table = ''.join([f'<tr><td>{k}</td><td>{v["name"]}</td><td>{v["age"]}</td><td>{v["location"]}</td><td>{v["photo"]}</td></tr>' for k, v in data.items() if category_filter in v["comment"]])
    title = category_filter
    timestamp = str(datetime.today().strftime('%d.%m.%Y %H:%M:%S'))
    header = r'<!DOCTYPE html>'+ '\n' + '<html>' + '\n' + '<head>' + '\n' + \
            '<meta http-equiv="content-type" content="text/html; charset=utf-8" />' + \
            '\n' + '<title>' + title + '</title>' + '\n' + '</head>' + '\n' + \
            '<style>a {text-decoration: none;} </style>' + '\n'
    content = f'{header}<body style="font-family:Arial">\n<b>{title}</b><br><br> \
        <table>{table}</table><br><br>{timestamp}'
    with codecs.open(export_filename, 'w', 'utf-8') as f:
        f.write(content)
    log.info(f'data printed: {export_filename}')

# change data
def change_data(id, target, value):
    global data
    value_new = value
    value_old = data[id][target]
    log.debug(f'data changing in {id} {target} from {value_old} to {value_new}...')
    data[id][target] = value_new
    log.debug(f'data changed: {id} {target} {value_old} => {value_new}')
    return 0

# filter data   
def filter_data():
    global data, profile_data_load, profile_photos_load, progress
    profile_data_load = True
    profile_photos_load = False
    script = []; source = ''; destination = ''; conditions = []
    filename = 'filter.script'
    if os.path.isfile(filename):
        log.info(f'data filtering by script {filename}...')
        with codecs.open(filename, 'r', 'utf-8') as f:
            script_source = f.read()
        script = [line for line in script_source.split('\n') if (line != '' and not line.startswith('#'))]
        source, destination, conditions = script[0].strip(), script[1].strip(), script[2:]
    else:
        log.error(f'data filtering canceled: script {filename} not found')
        return
    if source == '' or destination == '':
        log.error(f'filtering error: invalid source or destination values')
        return
    else:
        log.info(f'filtering script: {source}, {destination}, {conditions} ')
    if source.isdigit():
        ids = [source]
    else:
        ids = [k for k in data.keys() if data[k]['comment'] == source]
    db_size = [len(ids), len(ids)]
    if destination == 'NULL':
        for k, v in data.copy().items():
            if v['comment'] == source:
                data.pop(k, None)
        log.info(f'ALL * {source} => NULL')
    else:
        id_counter = 0
        id_max = min(len(ids), capacity)
        progress.min = 0
        progress.max = id_max
        for id in ids:
            if id_counter > id_max:
                break
            profile = profile_read(id)
            id_counter += 1
            progress.value = id_counter
            if profile['closed']:
                log.info(f'{id} closed -> PASSED\n')
                continue
            elif profile['inaccessible']:
                log.info(f'{id} closed -> PASSED\n')
                continue
            else:
                for condition in conditions:
                    try:
                        result = (eval(condition.strip()) == True)
                        if result:
                            change_data(id, 'comment', destination)
                            log.info(f"{id} {data[id]['name']}: {source} => {destination} ({condition})")
                            time.sleep(0)
                            break
                    except:
                        pass
        ids = [k for k in data.keys() if data[k]['comment'] == source]
        db_size[1] = len(ids)
        log.info(f'filtering complete: size of category DB {db_size[0]} => {db_size[1]} items')
        return db_size[0], db_size[1]

# update data
def update_data():
    global data, progress
    log.info(f'data updating to last {capacity} new items...')
    parsed_counter_max = capacity
    url = f'https://www.mamba.ru/{LANGUAGE}/search/list'
    content = ''
#     driver.implicitly_wait(10)
    log.debug(f'page {url} loading and identification...')
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[contains(@href, "search-settings")]')))
    log.debug(f'page {url} loaded OK')
    log.debug(f'search filter setting...')
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@href, "search-settings")]'))).click()
    # geo
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@href, "search-settings/geoSelect")]'))).click()
    time.sleep(1)
    if COUNTRY not in WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/form[@data-name="main-search-form-geoSelect"]'))).get_attribute('innerHTML'):
        log.error('Target detected No Russians. Setting geosetting not impossible')
        return -1, -1, -1, -1
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@data-name, "main-search-another-city-action")]'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/input[@name="location"]'))).clear()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/input[@name="location"]'))).send_keys(Keys.CONTROL + "A")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/input[@name="location"]'))).send_keys(Keys.DELETE)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/input[@name="location"]'))).send_keys(CITY)
    time.sleep(1)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/input[@name="location"]'))).send_keys(Keys.ARROW_DOWN)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/input[@name="location"]'))).send_keys(Keys.ENTER)
    time.sleep(1)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@data-name, "close-action")]'))).click()    
#      purpose
#     WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@data-name, "main-search-purpose-action")]'))).click()
#     time.sleep(1)
#     purposec = ['Friendship', 'Email', 'Love', 'Flirtation', 'Family', 'Sport', 'Travel', 'Meet']
#     for purpose in purposec:
#         if driver.find_element(By.XPATH, f'.//input[contains(@name, "{purpose}")]').get_attribute('checked'):
#             driver.find_element(By.XPATH, f'.//div[contains(@data-name, "{purpose}")]').click()
#     for purpose in ['Love', 'Family']:
#         if not driver.find_element(By.XPATH, f'.//input[contains(@name, "{purpose}")]').get_attribute('checked'):
#             driver.find_element(By.XPATH, f'.//div[contains(@data-name, "{purpose}")]').click()
#     time.sleep(1)
#     WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@data-name, "close-action")]'))).click()
    # age
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@data-name, "main-search-age-action")]'))).click()
    time.sleep(1)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/select[contains(@name, "ageFrom")]'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/select[contains(@name, "ageFrom")]'))).send_keys('25')
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/select[contains(@name, "ageTo")]'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/select[contains(@name, "ageTo")]'))).send_keys('38')
    time.sleep(1)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@data-name, "close-action")]'))).click()   
    # gender
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@href, "search-settings/gender")]'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/div[@id="F"]'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[contains(@data-name, "close-action")]'))).click()
    log.debug(f'search filter OK')
    time.sleep(1)
    log.debug(f'items scrolling...')
    scroll_counter_max = int (parsed_counter_max / 50)
    progress.max = scroll_counter_max
    scroll_counter = 0
    progress.value = scroll_counter
    driver.execute_script("window.scrollTo(top);")
    time.sleep(1)
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scroll_counter += 1
        progress.value = scroll_counter
        time.sleep(1)
        if content == driver.page_source or scroll_counter > scroll_counter_max:
            break
        content = driver.page_source
    time.sleep(1)
    log.debug(f'items scrolled: {scroll_counter} times')
    log.debug(f'items identification...')
    items = WebDriverWait(driver, 30, ignored_exceptions=ignored_exceptions).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="grid-container"]//div[@data-name="search-list-item"]')))
    identification_counter = len(items)
    log.debug(f'identified items: {identification_counter}')
    log.debug(f'items parsing...')
    progress.max = parsed_counter_max
    parsed_counter = 0
    progress.value = parsed_counter
    updated_counter = 0
    passed_counter = 0
    load_data()
    for item in items:
        id = ''; name = ''; age =''; location = ''; photo = '';
        with suppress(ignored_exceptions):
            id = item.find_element(By.XPATH, './/a').get_attribute('href').split('/')[5]
            if data.get(id):
                passed_counter += 1
            else:
                name = item.find_element(By.XPATH, './/span[contains(@data-name, "item-title-name")]').text
                age = item.find_element(By.XPATH, './/div[@data-name="item-title-age"]').text
                location = item.find_element(By.XPATH, './/span[@data-name="space-time-location-text"]').text
                photo_src = item.find_element(By.XPATH, './/img[@data-name="user-photo"]').get_attribute('src')
                photo = f'<a href="https://www.mamba.ru/{LANGUAGE}/profile/{id}" target="_blank">{embeded_image(photo_src)}</a>'
                comment = categories[0]
                data[id] = {'name' : name, 'age' : age, 'location' : location, 'photo' : photo, 'comment' : comment}
                updated_counter += 1
        parsed_counter += 1
        progress.value = parsed_counter
        log.debug(f'data updating: identificated {identification_counter}, parsed {parsed_counter}, updated {updated_counter}, passed {passed_counter}')
        if parsed_counter >= parsed_counter_max:
            break
        time.sleep(0)
    log.info(f'data updated: identificated {identification_counter}, parsed {parsed_counter}, updated {updated_counter}, passed {passed_counter}')
    return identification_counter, parsed_counter, updated_counter, passed_counter

# messages box
def messages_box():
    global driver
    box = []
    log.info(f'new messages reading...')
    # open chat
    log.debug(f'messages page loading...')
    url = f'https://www.mamba.ru/contact/list'
    driver.get(url)
    items = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH, './/div[@class="ReactVirtualized__Grid__innerScrollContainer"]')))
    log.debug(f'messages page loaded')
    # messages links identification
    log.debug(f'items identification...')
    items = WebDriverWait(driver, 30, ignored_exceptions=ignored_exceptions).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="ReactVirtualized__Grid__innerScrollContainer"]//div[contains(@data-name, "contact-") and contains(@data-name, "-item")]')))
    identification_counter = len(items)
    log.debug(f'identified items: {identification_counter}')
    log.debug(f'items parsing...')
    parsed_counter = 0
    unread_counter = 0
    for item in items:
        id = ''; name = ''; unread = False; newmsg = '';
        with suppress(ignored_exceptions):
            id = item.find_element(By.XPATH, './/a[contains(@href, "contact")]').get_attribute('href').split('/')[-2]            
            name = item.find_element(By.XPATH, './/h3').text
            unread = bool(item.find_element(By.XPATH, './/div[@data-name="total-count"]').text)
            newmsg = item.find_element(By.XPATH, './/a[contains(@href, "contact")][2]/div').text
        if not id.isdigit():
            continue
        box.append([id, name, newmsg])
        if unread:
            log.info(f'new message from {id}: {newmsg}')
            if data.get(id):
                change_data(id, 'comment', categories[2])
                log.info(f'{id} moved => {categories[2]}')
            else:
                log.info(f'{id} not in DB')
            unread_counter += 1
        parsed_counter += 1
    log.debug(f'items parsed: {parsed_counter}')
    log.info(f'new messages readed: {unread_counter} messages')
    return box
    
def message_read(id):
    visited = f'анкету смотрела {guests[int(id)]}' if int(id) in guests else 'анкету возможно ещё не видела'
    log.info(f'message history {id} reading...')
    messages = []
    # open chat
    url = f'https://www.mamba.ru/chats/{id}/contact'
    log.debug(f'chat page loading...')
    driver.get(url)
    log.debug(f'chat page loaded')
    # read messages
    identification_counter = 0
    parsed_counter = 0
    log.debug(f'items identification...')
    last_visit = ''
    try:
        last_visit = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="app-wrapper"]/div[4]/div/div[2]/div/div/div[1]/div/a/div/div/div/div/div[1]'))).text.lower()
    except:
        pass
    if 'была' not in last_visit:
        last_visit = 'была неизвестно когда'
        try:
            if bool(driver.find_element(By.XPATH, './/span[@data-name="messenger-online-indicator"]')):
                last_visit = 'сейчас ОНЛАЙН'
        except:
            pass
        try:
            if bool(driver.find_element(By.XPATH, './/div[(contains(text(), "Анкета удалена"))]')):
                last_visit = 'анкета УДАЛЕНА'
        except:
            pass

    WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH, './/div[@data-name="messenger-layout"]')))
    # read messages
    try:
        items = WebDriverWait(driver, 2, ignored_exceptions=ignored_exceptions).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@data-name="messenger-layout"]//li/div')))
        identification_counter = len(items)
    except:
        identification_counter = 0
    if identification_counter > 0:
        for item in items:
            text = ''; date = ''; readed = False; unreaded = False; nodate = False; mark = ''; author = ''
            with suppress(ignored_exceptions):
                text = item.find_element(By.XPATH, './/span').text
                date = item.find_element(By.XPATH, './/div/div/span').text
            try:
                item.find_element(By.XPATH, './/div/span[@data-name="message-read-icon"]')
                readed = True
            except NoSuchElementException:
                readed = False
            try:
                item.find_element(By.XPATH, './/div/span[@data-name="message-unread-icon"]')
                unreaded = True
            except NoSuchElementException:
                unreaded = False
            if date != '':
                if not readed and not unreaded:
                    author = 'О'
                elif item == items[-1] and unreaded:
                    author = 'Я'
                    mark = f'\n[НЕ ПРОЧИТАНО]'
                elif item == items[-1] and readed:
                    author = 'Я'
                    mark = f'\n[ПРОЧИТАНО]'
                else:
                    author = 'Я'
                messages.append(f'{date} {author}: {text}{mark}')
            else:
                messages.append(f'[{text}]')
            parsed_counter += 1
    messages.append(f'\n[{last_visit}, {visited}]')
    log.debug(f'items parsed')

    log.debug(f'identified {identification_counter} items, parsed {parsed_counter} items')
    log.info(f'message history {id} readed: {parsed_counter} messages')
    return messages

def message_write(id, message):
    log.info(f'message writing to {id}...')
    # open chat
    url = f'https://www.mamba.ru/chats/{id}/contact'
    log.debug(f'chat page loading...')
    driver.get(url)
    items = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH, './/div[@data-name="messenger-layout"]')))
    log.debug(f'chat page loaded')
    # write message
    try:
        log.debug(f'sending message...')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/textarea[@data-name="input-textarea"]'))).clear()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/textarea[@data-name="input-textarea"]'))).send_keys(Keys.CONTROL + "A")
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/textarea[@data-name="input-textarea"]'))).send_keys(Keys.DELETE)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/textarea[@data-name="input-textarea"]'))).send_keys(message)
        time.sleep(1)
        button_send = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@data-name="messenger-send-message-icon"]')))
        driver.execute_script("arguments[0].click();", button_send)
        change_data(id, 'comment', categories[2])
        log.info(f'{id} moved => {categories[2]}')
        log.info(f'message writed to {id}: {message}')
        return f'message sended ({id} automatically moved to category {categories[2]})'
    except:
        log.error(f'message not writed to {id}: {message}')
        return 'message not sended'

def profile_read(id, limit_photos=3):
    global driver, data, profile_cache
    log.info(f'profile {id} reading...')
    if (profile_cache != [] and id == profile_cache['id']) and (not profile_photos_load or (profile_photos_load and profile_cache['photos'] != [])):
        log.debug(f'profile {id} readed caching: {profile_cache}')
        return profile_cache
    url = f'https://www.mamba.ru/{LANGUAGE}/profile/{id}'
    # open profile
    log.debug(f'profile page loading...')
    driver.get(url)
    time.sleep(1)
    log.debug(f'profile page loaded')
    log.debug(f'profile status checking...')
    profile = {'closed' : False, 'blocked' : False, 'ignored' : False, 'inaccessible' : False}
    with suppress(ignored_exceptions):
        profile['closed'] = (anonymous_mode and bool(driver.find_element(By.XPATH, './/button[@data-name="gender-action"]')))
    with suppress(ignored_exceptions):
        profile['blocked'] = bool(driver.find_element(By.XPATH, './/div[@data-name="uninotice-title-profile-not-found"]'))
    with suppress(ignored_exceptions):
        profile['ignored'] = bool(driver.find_element(By.XPATH, './/div[@data-name="uninotice-title-user-ignored-you"]'))
    with suppress(ignored_exceptions):
        profile['inaccessible'] = (bool(driver.find_element(By.XPATH, './/div[@data-name="uninotice-title-hit-list-first-limit-exceeded"]')) or not bool(driver.find_element(By.XPATH, './/section')))
    log.debug(f'profile {id} status checked: {profile}')
    if profile['blocked'] or profile['closed'] or profile['ignored'] or profile['inaccessible']:
        log.info(f'profile readed: {profile}')
        return profile
    # read data
    profile_data = dict.fromkeys(["profile-location", "profile-greeting", "look-for", "look-for-age-range", "dating-goals", "profile-interests", "home-status", "children", "education", "known-languages", "smoking-attitude", "alcohol-attitude", "profile-countries", "appearance", "constitution", "height", "weight", "orientation", "horoscope-sign"])
    log.debug(f'items parsing...')
    for key, value in profile_data.items():
        with suppress(ignored_exceptions):
            profile_data[key] = driver.find_element(By.XPATH, f'.//div[@data-name="{key}"]').text
    profile_data["profile-location"] = data[id]['location']
    log.debug(f'items parsed')
    profile['data'] = profile_data
    log.info(f'profile {id} data readed: {profile["data"]}')
    log.debug(f'photos parsing...')
    profile['photos'] = []
    if profile_photos_load:
        photos = []
        photoviewer = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'.//a[contains(@href, id) and contains(@data-name, "photoviewer-") and contains(@data-name, "-action")]')))
        photoviewer.click()
        counter, total = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@data-name="photos-counter-component"]'))).text.split(' из ')
        for index in range(int(counter), int(total)+1):
            time.sleep(1)
            link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@data-name="photoviewer-wrapper"][1]/div[contains(@data-name, "photos-")][2]/img'))).get_attribute('src')
            photos.append(link)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[@data-name="arrow-right"]'))).click()
            if index >= limit_photos:
                break
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[@data-name="modal-photoviewer-close-button"]'))).click()
        profile['photos'] = [embeded_image(link) for link in photos]
        log.debug(f'photos parsed')
        log.info(f'profile {id} photos readed: {len(profile["photos"])} items')
    profile_cache = profile
    profile_cache['id'] = id
    return profile

def communicate(id):
    log.debug(f'communicator loading...')
    name = ''; history = ''; info = ''; photos = []
    if id:
        if profile_history_load:
            if anonymous_mode:
                history = 'NOT AUTHORIZATED'
            else:
                history = '\n'.join(msg for msg in message_read(id))
        if profile_data_load:
            profile = profile_read(id)
            if profile['closed'] or profile['blocked'] or profile['ignored'] or profile['inaccessible']:
                info = f'{profile}'
            else:
                for key, value in profile['data'].items():
                    if value == None:
                        value = '?'
                    elif '\n' in str(value):
                        value = value.replace('\n', ', ')
                    info += f'{key}: {value:25}\n'
                if profile_photos_load:
                    photos = profile['photos']
        name = data[id]['name']
    display_dialog(id, name, info, photos, history)
    log.debug(f'communicator loaded')

def guests_box(only_recents=False):
    global driver, guests
    log.info(f'guest checking...')
    # open visites
    log.debug(f'visites page loading...')
    url = f'https://www.mamba.ru/event-list/visits'
    driver.get(url)
    items = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH, './/div[@id="guaranteed-promo"]')))
    log.debug(f'visites page loaded')
    # messages scrolling
    log.debug(f'items scrolling...')
    if only_recents:
        scroll_counter_max = 0
    else:
        scroll_counter_max = 500
    scroll_counter = 0
    content = ''
    driver.execute_script("window.scrollTo(top);")
    time.sleep(0)
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scroll_counter += 1
        time.sleep(1)
        if content == driver.page_source or scroll_counter > scroll_counter_max:
            break
        content = driver.page_source
    time.sleep(1)
    log.debug(f'items scrolled: {scroll_counter} times')
    # messages links identification
    log.debug(f'items identification...')
    items = WebDriverWait(driver, 30, ignored_exceptions=ignored_exceptions).until(EC.presence_of_all_elements_located((By.XPATH, '//a[@data-name="link-hitlist-item-action"]')))
    identification_counter = len(items)
    log.debug(f'identified items: {identification_counter}')
    log.debug(f'items parsing...')
    parsed_counter = 0
    for item in items:
        id = ''; date = '';
        with suppress(ignored_exceptions):
            id = item.get_attribute('href').split('/')[-1].split('?')[0]
            date = item.find_element(By.XPATH, './/span[@data-name="hitlist-date-text"]').text
            parsed_counter += 1
        if id.isdigit():
            guests[int(id)] = date
        else:
            continue            
    log.debug(f'items parsed: {parsed_counter}')
    log.info(f'guest checked')
    return len(guests)

def say_hello(category = 'Отложенные', delay=30):
    global progress, hello_msgs
    if anonymous_mode:
        display_outbox('NOT AUTHORIZATED')
        return
    ids = [k for k, v in data.items() if category in v["comment"]]
    progress.min = 0
    progress.max = len(ids)
    id_counter = 0
    for id in ids:
        msg = choice(hello_msgs).capitalize() + ', ' + data[id].get('name')
        if message_write(id, msg) == 'message not sended':
            break
        id_counter += 1
        progress.value = id_counter
        time.sleep(delay)
    save_data()

# edit data
def command_change(change):
    change_data(change.owner.description_tooltip, 'comment', change.new) 
def command_capacity(change):
    global capacity
    capacity = int(change.new)
def command_message(change):
    global current_message_text
    current_message_text = change.new
def command_message_send(change):
    id = change.tooltip
    if not anonymous_mode:
        result = message_write(id, current_message_text)
        display_outbox(result, 1)
        communicate(id)
    else:
        display_outbox('ANONYMOUS MODE', 2)
def command_messages_hello(change):
    if anonymous_mode:
        display_outbox('NOT AUTHORIZATED')
    else:
        say_hello()
        display_outbox('hello finished')
def command_info(change):
    global profile_data_load
    profile_data_load = change.new
def command_photos(change):
    global profile_photos_load
    profile_photos_load = change.new
def command_history(change):
    global profile_history_load
    profile_history_load = change.new
def command_print(change):
    display_outbox('data printing...')
    print_data(category_filter = change.tooltip)
    display_outbox('data printed')
def command_save(change):
    display_outbox('data saving...')
    save_data()
    display_outbox('data saved')
def command_load(change):
    display_outbox('data loading...')
    load_data()
    display_outbox('data loaded')
def command_update(change):
    global anonymous_mode
    if not anonymous_mode:
        anonymous_mode = True
        main()
    display_outbox('data updating...')
    identification_counter, parsed_counter, updated_counter, passed_counter = update_data()
    display_outbox('data filtering...')
    inbox_counter, filtered_counter = filter_data()
    display_outbox('data saving...')
    save_data()
    print_data(category_filter = 'Входящие')
    anonymous_mode = False
    main()
    display_outbox(f'data updated: identificated {identification_counter}, parsed {parsed_counter}, updated {updated_counter}, passed {passed_counter}\ndata filtered: {inbox_counter} => {filtered_counter}')
def command_clear(change):
    source = change.tooltip
    destination = categories[3]
    moveto_data(source, destination)
    display_outbox(f'All elements moved from {source} to {destination}')
def command_communicate(change):
    id = change.description
    display_outbox('communicate loading...')
    communicate(id)
def command_offline_mode(change):
    global offline_mode
    offline_mode = change.new
    main()
def command_anonymous_mode(change):
    global anonymous_mode
    anonymous_mode = change.new
    main()
def command_DB_filename(change):
    global DB_filename
    DB_filename = change.new
    
def display_dialog(id, name, info, photos, history):
    global outbox
    history_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width='900px')
    history_label = Label(description='History', value=f'Dialog #{id} {name}', layout=Layout(width='90%'))
    history_textarea = Textarea(description='History', value=history, layout=Layout(width='90%', height='200px'))
    message_text = Text(description='Message', layout=Layout(width='90%'))
    message_text.observe(command_message, 'value')
    send_button = Button(description='Send', tooltip=id)
    send_button.on_click(command_message_send)
    history_box = Box(children=[history_label, history_textarea, message_text, send_button], layout=history_layout)
    info_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width='900px')
    info_label = Label(description='info', value=f'Info #{id} {name}', layout=Layout(width='90%'))    
    info_textarea = Textarea(description='Info', value=info, layout=Layout(width='90%', height='200px'))
    info_box = Box(children=[info_label, info_textarea], layout=info_layout)
    photos_gridbox =[]
    for pic in photos:
        photos_gridbox.append(GridBox(children=[HTML(value=pic, layout=Layout(width='auto', grid_area='photo'))],
            layout=Layout(
                height='400px',
                min_width='300px',
                grid_template_rows='auto',
                grid_template_columns='100%',
                grid_template_areas='''
                "photo"
                ''')
               )
        )
    photos_layout = Layout(overflow='scroll hidden',
                        border='1px solid black',
                        width='900px',
                        height='',
                        flex_flow='row',
                        display='flex')
    photos_box = Box(children=photos_gridbox, layout=photos_layout)
    with outbox:
        clear_output(True)
        if profile_data_load:
            display(info_box)
        if profile_photos_load:
            display(photos_box)
        if profile_history_load:
            display(history_box)

def display_outbox(content, interval=0):
    global outbox
    message = Textarea(description='Outbox', value=f'{datetime.now().strftime(date_format):}\n{content}', layout=Layout(width='90%', height='200px'))
    box = Box(children=[message], layout=Layout(display='flex', flex_flow='column', align_items='stretch', width='900px'))
    with outbox:
        clear_output(True)
        display(box)
    if interval != 0:
        time.sleep(interval)
        outbox.clear_output()
        
def UI(category_filter=''):
    global data, outbox
    global outbox
    log.info(f'items displaying...')
    # items controls
    profile_items = []
    display_counter = 0
    for k, v in data.items():
        if category_filter == v['comment']:
            button_id = Button(description=k, disabled=offline_mode, layout=Layout(width='auto', grid_area='id'))
            button_id.on_click(command_communicate)
            html_name = HTML(value=v['name'], layout=Layout(width='auto', grid_area='name'))
            html_age = HTML(value=str(v['age']), layout=Layout(width='auto', grid_area='age'))
            html_location = HTML(value=v['location'], layout=Layout(width='auto', grid_area='location'))
            html_photo = HTML(value=v['photo'], layout=Layout(width='auto', grid_area='photo'))
            select_comment = Select(value=v['comment'], description_tooltip=str(k), options=categories, continuous_update=False, layout=Layout(width='auto', grid_area='comment'))
            select_comment.observe(command_change, 'value')
            profile_items.append(GridBox(children=[button_id, html_name, html_age, html_photo, select_comment],
                layout=Layout(height='400px', min_width='300px', grid_template_rows='auto auto auto', grid_template_columns='45% 45% 10%',
                    grid_template_areas='''
                    "photo photo photo"
                    "id name age"
                    "comment comment comment"
                    ''')))
            display_counter += 1
    profile_layout = Layout(overflow='scroll hidden', border='3px solid black', width='900px', height='', flex_flow='row', display='flex')
    profile_items.reverse()
    profile_box = Box(children=profile_items, layout=profile_layout)
    # categories control
    statistics = ', '.join([f'{category}: {len([k for k, v in data.items() if category in v["comment"]])}' for category in categories])
    html_statistics = HTML(value=f'Total elements: {display_counter} ({statistics})')    
    button_clear = Button(description='Clear', tooltip=category_filter)
    button_clear.on_click(command_clear)
    # common data controls
    button_print = Button(description='Print', tooltip=category_filter)
    button_print.on_click(command_print)
    button_save = Button(description='Save',)
    button_save.on_click(command_save)
    button_load = Button(description='Load',)
    button_load.on_click(command_load)
    dropdown_capacity = Dropdown(options=['10', '100', '200', '500', '1000', '2000', '5000'], value=str(capacity), description='', disabled=False, layout=Layout(width='100px'))
    dropdown_capacity.observe(command_capacity, 'value')
    button_update = Button(description='Update', disabled=offline_mode, )
    button_update.on_click(command_update)
    button_hello = Button(description='Hello',  disabled=offline_mode or anonymous_mode, )
    button_hello.on_click(command_messages_hello)
    # profile data controls
    checkbox_profile_info = Checkbox(description='Load profile info', value=profile_data_load)
    checkbox_profile_info.observe(command_info, 'value')
    checkbox_profile_photos = Checkbox(description='Load profile photos', value=profile_photos_load)
    checkbox_profile_photos.observe(command_photos, 'value')
    checkbox_profile_history = Checkbox(description='Load profile history', value=profile_history_load)
    checkbox_profile_history.observe(command_history, 'value')
    # common settings controls
    label_settings_title =  Label(description='Settings', value=f'Settings', layout=Layout(width='90%'))
    checkbox_settings_offline_mode = Checkbox(description='offline_mode', value=offline_mode)
    checkbox_settings_offline_mode.observe(command_offline_mode, 'value')
    checkbox_settings_anonymous_mode = Checkbox(description='anonymous_mode', value=anonymous_mode)
    checkbox_settings_anonymous_mode.observe(command_anonymous_mode, 'value')
    text_settings_DB_filename = Text(description='DB_filename', value=DB_filename, layout=Layout(width='90%'))
    text_settings_DB_filename.observe(command_DB_filename, 'value')
    layout_settings = Layout(overflow='scroll hidden', border='1px solid black', width='900px', height='', flex_flow='column', display='flex')
    box_settings = HBox(children=[label_settings_title, checkbox_settings_offline_mode, checkbox_settings_anonymous_mode, text_settings_DB_filename], layout=layout_settings)
    display(
        HBox([html_statistics, button_clear]),
        HBox([button_print, button_save, button_load, dropdown_capacity, button_update, button_hello]),
        HBox([checkbox_profile_info, checkbox_profile_photos, checkbox_profile_history]),
        HBox([profile_box]),
        HBox([progress]),
        VBox([outbox]),
        VBox([box_settings]),
    )
    log.info(f'displayed: {display_counter} items')

def main():
    global data, ui, offline_mode, anonymous_mode
    ip = []
    log.info(f'starting...')
    clear_output(True)
    with outbox:
        clear_output(True)
    display_outbox(f'LOADING DB...')
    load_data()
    display_outbox(f'RENDERING UI...')
    ui = interactive(UI, category_filter=categories);
    display(ui)
    display_outbox(f'CONNECTING...')
    if offline_mode:
        display_outbox(f'OFFLINE MODE')
    else:
        ip = check_ip()
        drive()
        if not anonymous_mode:
            anonymous_mode = not authorization()
        if anonymous_mode:
            display_outbox(f'ONLINE MODE {ip} (not authorizated)')
        else:
            display_outbox(f'ONLINE MODE {ip}')
            guests_box()
            new_messages = 'messages checked\n\n' + '\n'.join(f'{msg[0]} {msg[1]}: {msg[2]}' for msg in messages_box() if msg[2] != '')
            display_outbox(new_messages)

    log.info(f'started: {DB_filename}, {offline_mode}, {anonymous_mode}, {ip}')

main()

In [ ]:
# clean deleted profiles script (В работе -> Отложенные)
guests_box()
source = 'В работе'
destination = 'Отложенные'
conditions = ['анкета УДАЛЕНА', 'была неизвестно когда, анкету возможно ещё не видела']
ids = [k for k, v in data.items() if source in v["comment"]]
for id in ids:
    status = ''
    try:
        status = message_read(id)[-1].split('\n')[-1]
    except:
        pass
    for condition in conditions:
        if condition in status:
            change_data(id, 'comment', destination)
            log.info(f"{id} {data[id]['name']}: {source} => {destination} ({condition})")
            time.sleep(0)
            break

In [ ]:
# fix bug proxy_default?hit=35
target = 'proxy_default'
space = 'https://www.mamba.ru/profile/'
result = dict()
for k, v in data.items():
    if target in str(v.items()):
        for x, y in v.items():
            if target in str(y):
                result[k] = {x : y}
                space_start = y.index(space)
                space_end = y.index('/', y.index(space) + len(space))
                url_end = y.index('"', space_end)
                old = y[space_start:url_end]
                new = y[space_start:space_end]
                data[k][x] = y.replace(old, new)
                print(f'{old} => {new}')